In [ ]:
!pip3 install requests

In [ ]:
import requests 
import random
import string
import json
from datetime import datetime

base_url = "https://du-backend-t437.onrender.com"
# base_url = "http://localhost:8002"

ir_ids = [
    "IM7216", "IM6599", "IM1579", "IM9252", "IM2004", "IM4230", "IM7990", "IM0262",
    "IM1781", "IM1332", "IM9002", "IM4546", "IM4927", "IM6392", "IM2779", "IM9215",
    "IM8947", "IM8361", "IM2499", "IM0424", "IM6678", "IM9375", "IM8657", "IM5852", "IM1530"
]

In [ ]:

def createIr(new_id):  # Make sure this is defined
    path = "/api/add_ir_id"
    end_point = f"{base_url}{path}"
    payload = {"ir_id": new_id}

    try:
        response = requests.post(end_point, json=payload, timeout=10)
        response.raise_for_status()
        print(response.json())
    except requests.exceptions.Timeout:
        print("Request timed out.")
    except requests.exceptions.ConnectionError:
        print("Failed to connect to the server.")
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err.response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")


for ir_id in ir_ids:
    createIr(ir_id)


In [ ]:
access_distribution = (
    [1] +                  # Admin (1 user)
    [2] * 5 +              # LDC (5 users)
    [3] * 8 +              # LS (8 users)
    [4] * (len(ir_ids) - (1 + 5 + 8))  # GC (Remaining users)
)

def register_ir(ir_id, access_level):
    path = "/api/register_new_ir"
    end_point = f"{base_url}{path}"
    data = {
        "ir_id": ir_id,
        "ir_name": f"Test User {ir_id}",
        "ir_email": f"{ir_id.lower()}@example.com",
        "ir_password": "TestPass123",
        "ir_access_level": access_level
    }
    
    try:
        response = requests.post(
            end_point,
            json=data,
            timeout=10
        )
        
        if response.ok:
            print(f"✅ Registered {ir_id} (Access: {access_level})")
        else:
            try:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} | {response.json()}")
            except ValueError:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} | {response.text}")
    
    except requests.exceptions.RequestException as e:
        print(f"🚫 Failed {ir_id}: {e}")

def populate_test_irs():
    for ir_id, access_level in zip(ir_ids, access_distribution):
        register_ir(ir_id, access_level)


populate_test_irs()

In [ ]:
def getAllRegisteredIr():
    path = "/api/irs"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    
    if response.ok:
        try:
            print("Result:", response.json())
        except ValueError:
            print("Result (non-JSON):", response.text)
    else:
        try:
            print("Error:", response.json())
        except ValueError:
            print("Error (non-JSON):", response.text)

getAllRegisteredIr()


In [ ]:
def createTeam():
    path = "/api/create_team"
    end_point = f"{base_url}{path}"
    data = {"name":"Wolfpacks"}
    data = json.dumps(data)
    response = requests.post(end_point,data=data,headers={"Content-type":"application/json"})
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

createTeam()

In [ ]:
def getAllTeams():
    path = "/api/teams"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

getAllTeams()

In [ ]:
def update_team_name(team_id, new_name):
    path = f"/api/update_team_name/{team_id}"  # <-- f-string here
    end_point = f"{base_url}{path}"
    payload = {"name": new_name}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.patch(end_point, json=payload, headers=headers)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

update_team_name(3, "Dreame Relentless")

In [ ]:
def deleteTeam(team_id):
    path = f"/api/delete_team/{team_id}"
    end_point = f"{base_url}{path}"
    response = requests.delete(end_point)
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

deleteTeam(1)

In [ ]:
def add_ir_to_team(ir_id, team_id, role):
    url = f"{base_url}/api/add_ir_to_team"
    payload = {
        "ir_id": ir_id,
        "team_id": team_id,
        "role": role
    }
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Example usage:

import random

def assign_irs_to_teams(ir_ids, team_ids, ir_access_map):
    """
    Assigns 1 LDC, 1 LS, and 1 GC/IR to each team.
    """
    random.shuffle(ir_ids)

    # Separate IRs by access level
    ldc_ids = [ir for ir in ir_ids if ir_access_map[ir] == 2]
    ls_ids = [ir for ir in ir_ids if ir_access_map[ir] == 3]
    gc_ids = [ir for ir in ir_ids if ir_access_map[ir] == 4]
    ir_only_ids = [ir for ir in ir_ids if ir_access_map[ir] not in [2, 3, 4]]

    assignments = []

    for i, team_id in enumerate(team_ids):
        # Assign LDC
        if i < len(ldc_ids):
            assignments.append((ldc_ids[i], team_id, "LDC"))
        else:
            print(f"Not enough LDCs for team {team_id}")

        # Assign LS
        if i < len(ls_ids):
            assignments.append((ls_ids[i], team_id, "LS"))
        else:
            print(f"Not enough LSs for team {team_id}")

        # Assign GC (or IR if not enough GCs)
        if i < len(gc_ids):
            assignments.append((gc_ids[i], team_id, "GC"))
        elif i < len(ir_only_ids):
            assignments.append((ir_only_ids[i], team_id, "IR"))
        else:
            print(f"Not enough GC/IRs for team {team_id}")

    return assignments

# Example usage:
team_ids = [4, 5, 6]
ir_access_map = {
    "IM7216": 2, "IM6599": 3, "IM1579": 3, "IM9252": 2, "IM2004": 4, "IM4230": 4, "IM7990": 4, "IM0262": 4,
    "IM1781": 3, "IM1332": 3, "IM9002": 2, "IM4546": 3, "IM4927": 4, "IM6392": 4, "IM2779": 4, "IM9215": 4,
    "IM8947": 4, "IM8361": 4, "IM2499": 4, "IM0424": 4, "IM6678": 4, "IM9375": 4, "IM8657": 4, "IM5852": 4, "IM1530": 4
}
ir_ids = list(ir_access_map.keys())

assignments = assign_irs_to_teams(ir_ids, team_ids, ir_access_map)

for ir_id, team_id, role in assignments:
    add_ir_to_team(ir_id, team_id, role)  # Call your API function here

for ir_id, team_id, role in assignments:
    add_ir_to_team(ir_id, team_id, role)  # Call your API function here

In [ ]:
def get_team_members(team_id):
    url = f"{base_url}/api/team_members/{team_id}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

get_team_members(6) 

In [ ]:
def remove_ir_from_team(base_url, team_id, ir_id):
    url = f"{base_url}/api/remove_ir_from_team/{team_id}/{ir_id}"
    try:
        response = requests.delete(url, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

remove_ir_from_team(base_url, 1, "IM1332")

In [ ]:

def add_info_detail(base_url, ir_id, info_date, response, comments, info_name):
    url = f"{base_url}/api/add_info_detail/{ir_id}"
    payload = {
        "ir_id": ir_id,
        "info_date": info_date,  # Should be ISO format string, e.g. "2025-08-26T12:00:00+05:30"
        "response": response,    # Should be one of "A", "B", "C"
        "comments": comments,
        "info_name": info_name
    }
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Example usage:
ir_id = "IM9252"
info_date = datetime.now().isoformat()  # Or use your preferred datetime string
response_option = "A"                   # Must be "A", "B", or "C"
comments = "Football tourment"
info_name = "Kiran A"

add_info_detail(base_url, ir_id, info_date, response_option, comments, info_name)

In [ ]:
def add_info_details(base_url, ir_id, info_list):
    url = f"{base_url}/api/add_info_detail/{ir_id}"
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, json=info_list, headers=headers, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Example usage:
ir_id = "IM7216"

info_list = [
    {
        "info_date": datetime.now().isoformat(),
        "response": "A",
        "comments": "First info went well.",
        "info_name": "John Doe"
    },
    {
        "info_date": datetime.now().isoformat(),
        "response": "B",
        "comments": "Second info, some issues.",
        "info_name": "Jane Smith"
    },
        {
        "info_date": datetime.now().isoformat(),
        "response": "A",
        "comments": "Second info, some issues.",
        "info_name": "Smith"
    },
    {
        "info_date": datetime.now().isoformat(),
        "response": "B",
        "comments": "Second info, some issues.",
        "info_name": "Jenith"
    }
]

add_info_details(base_url, ir_id, info_list)

In [ ]:
import random
from datetime import datetime

def add_info_details(base_url, ir_id, info_list):
    url = f"{base_url}/api/add_info_detail/{ir_id}"
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(url, json=info_list, headers=headers, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Random data generators
responses = ["A", "B", "C"]
comments_list = [
    "Great progress!", "Needs improvement.", "All targets met.", "Follow-up required.", "Excellent work!"
]
info_names = ["John Doe", "Jane Smith", "Alex Lee", "Chris Kim", "Pat Taylor", "Morgan Ray"]

# Populate for all IRs
for ir_id in ir_ids:
    info_list = []
    for _ in range(random.randint(2, 5)):  # 2 to 5 info entries per IR
        info_list.append({
            "info_date": datetime.now().isoformat(),
            "response": random.choice(responses),
            "comments": random.choice(comments_list),
            "info_name": random.choice(info_names)
        })
    add_info_details(base_url, ir_id, info_list)

In [ ]:
def getAllInfoData(ir_id):
    path = f"/api/info_details/{ir_id}"
    end_point = f"{base_url}{path}"
    try:
        response = requests.get(end_point, timeout=10)
        response.raise_for_status()
        try:
            print("Result:", response.json())
        except ValueError:
            print("Result (non-JSON):", response.text)
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

getAllInfoData("IM7216")

In [ ]:
def get_info_details(base_url, ir_id, from_date=None, to_date=None):
    url = f"{base_url}/api/info_details/{ir_id}"
    params = {}
    if from_date:
        params["from_date"] = from_date
    if to_date:
        params["to_date"] = to_date

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        try:
            print(f"HTTP error occurred: {http_err} - {response.json()}")
        except Exception:
            print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

# Example usage:
ir_id = "IM7216"
get_info_details(base_url, ir_id,"2025-09-09","2025-09-10")  # Optionally add from_date, to_date

In [ ]:
def update_ir_info_data(info_id):
    path = f"/api/update_info_detail/{info_id}"
    end_point = f"{base_url}{path}"
    payload = {"info_name":"Snehith","comments":"Updated via script","response":"A"}
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.put(end_point, json=payload, headers=headers)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

update_ir_info_data(3)

In [ ]:
def delete_info(info_id):
    path =f"/api/delete_info_detail/{info_id}"
    end_point = f"{base_url}{path}"
    try:
        response = requests.delete(end_point, timeout=10)
        response.raise_for_status()
        print("Success:", response.json())
        return response.json()
    except Exception as e:
        print(f"An error occurred: {e}")

delete_info(3)

In [ ]:
def getLdc():
    path = "/api/ldcs"
    end_point =f"{base_url}{path}"
    response = requests.get(end_point)
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

getLdc()

In [ ]:
def getLdcTeamsManaged():
    path = "/api/teams_by_ldc/IM7216"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    if response.ok:
        print("Success ", response.json())
    else:
        print("Failure ", response.json())

getLdcTeamsManaged()

In [ ]:
import requests
import random

# def set_targets(base_url, acting_ir_id, team_id=None, team_weekly_info_target=None, team_weekly_plan_target=None, ir_id=None, weekly_info_target=None, weekly_plan_target=None):
#     url = f"{base_url}/api/set_targets"
#     payload = {
#         "acting_ir_id": acting_ir_id,
#         "team_id": team_id,
#         "team_weekly_info_target": team_weekly_info_target,
#         "team_weekly_plan_target": team_weekly_plan_target,
#         "ir_id": ir_id,
#         "weekly_info_target": weekly_info_target,
#         "weekly_plan_target": weekly_plan_target
#     }
#     payload = {k: v for k, v in payload.items() if v is not None}
#     headers = {"Content-Type": "application/json"}
#     try:
#         response = requests.put(url, json=payload, headers=headers, timeout=10)
#         response.raise_for_status()
#         print("Success:", response.json())
#     except Exception as e:
#         print("Error:", e, getattr(e, "response", None))

def set_targets(base_url, acting_ir_id, team_id=None, team_weekly_info_target=None, team_weekly_plan_target=None, ir_id=None, weekly_info_target=None, weekly_plan_target=None):
    url = f"{base_url}/api/set_targets"
    payload = {
        "acting_ir_id": acting_ir_id,
        "team_id": team_id,
        "team_weekly_info_target": team_weekly_info_target,
        "team_weekly_plan_target": team_weekly_plan_target,
        "ir_id": ir_id,
        "weekly_info_target": weekly_info_target,
        "weekly_plan_target": weekly_plan_target
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)  # <-- use POST
        response.raise_for_status()
        print("Success:", response.json())
    except Exception as e:
        print("Error:", e, getattr(e, "response", None))

# Get all LDCs
ldcs = requests.get(f"{base_url}/api/ldcs").json()
for ldc in ldcs:
    ldc_id = ldc["ir_id"]
    # Get teams managed by this LDC
    teams = requests.get(f"{base_url}/api/teams_by_ldc/{ldc_id}").json()
    for team in teams:
        team_id = team["id"]
        # Set team targets (random example values)
        team_weekly_info_target = random.randint(20, 40)
        team_weekly_plan_target = random.randint(10, 20)
        set_targets(
            base_url,
            acting_ir_id=ldc_id,
            team_id=team_id,
            team_weekly_info_target=team_weekly_info_target,
            team_weekly_plan_target=team_weekly_plan_target
        )
        # Get LSs in this team
        members = requests.get(f"{base_url}/api/team_members/{team_id}").json()
        ls_members = [m for m in members if m["role"] == "LS"]
        # Split team targets among LSs
        if ls_members:
            info_per_ls = team_weekly_info_target // len(ls_members)
            plan_per_ls = team_weekly_plan_target // len(ls_members)
            for ls in ls_members:
                set_targets(
                    base_url,
                    acting_ir_id=ldc_id,
                    ir_id=ls["ir_id"],
                    weekly_info_target=info_per_ls,
                    weekly_plan_target=plan_per_ls
                )

Error: 404 Client Error: Not Found for url: https://du-backend-t437.onrender.com/api/set_targets <Response [404]>
Error: 404 Client Error: Not Found for url: https://du-backend-t437.onrender.com/api/set_targets <Response [404]>
Error: 404 Client Error: Not Found for url: https://du-backend-t437.onrender.com/api/set_targets <Response [404]>
Error: 404 Client Error: Not Found for url: https://du-backend-t437.onrender.com/api/set_targets <Response [404]>
Error: 404 Client Error: Not Found for url: https://du-backend-t437.onrender.com/api/set_targets <Response [404]>
Error: 404 Client Error: Not Found for url: https://du-backend-t437.onrender.com/api/set_targets <Response [404]>
